In [1]:
import numpy as np
import h5py
from einops import rearrange
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import datetime

print(torch.__version__)

2.0.0+cu117


In [2]:
data_path = "./Combustion_data_ME592_Assignment/Aditya_data/combustion_img_13.mat"
arrays = {}
image_shape = (250, 100)



data = h5py.File(data_path)

for k, v in data.items():
    arrays[k] = np.array(v)

train_set_x = arrays["train_set_x"]
train_set_y = arrays["train_set_y"]

print("The shape of train x: ", train_set_x.shape)
print("The shape of train y: ", train_set_y.shape)
train_images = rearrange(train_set_x, "(y h) (x w) -> (y x) h w", w = image_shape[0], h = image_shape[1])

print("shape of train array: ", train_images.shape)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = './Combustion_data_ME592_Assignment/Aditya_data/combustion_img_13.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Define the CNN architecture
# now that our data preparation is complete, we can start setting up our CNN
# Input: Image
# Output: Stable/Unstable (1 or 0)

# these are the hyperparameters for the model
# remember that a batch is how many items the  model will train on before updating the model
batch_size = 4
# remember that an epoch is the number of times that the trainer will go through the entire dataset
epochs = 1 
# since our image is rectangular, let's use a rectangular kernel. 
# this touple is comprised of factors of the dimensions. 
kernel_size = (25, 10) 
input_height = train_images[0].shape[0]
input_width = train_images[0].shape[1]


stride = 1
classes = ("stable", "unstable")
number_channels = 1 # this is a grayscale image
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# to increase the speed of the training, make sure that the GPU is being used. 
print(torch.cuda.is_available())


print("Number of epochs: ", epochs)
print("Batch size: ", batch_size)
print("Kernel size: ", kernel_size)
print("Stride size: ", stride)
print("Classes: ", classes)
print("input height: ", input_height)
print("input width: ", input_width)
print("device type: ", device)
# print("The device being used is: ", device)

# These links were used for reference 
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
# https://www.analyticsvidhya.com/blog/2019/10/building-image-classification-models-cnn-pytorch/
# for a test, let's train this first without using the validation set. 
# that way we can prove that the validation set actually does help increase accuracy and decrease overfitting
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        # self.fc1 = nn.Linear(800, 64)
        self.fc1 = nn.Linear(89792, 1000)
        self.fc2 = nn.Linear(1000, 1)


    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))

        # num_flat_features = x.size(1)*x.size(2)*x.size(3)
        # x = x.view(-1, num_flat_features)

        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [ ]:
# we need to convert our numpy arrays into pytorch tensors
batch_size = 16
train_images_torch = torch.div(torch.from_numpy(train_images).float(), 255)
print("train_images.size: ", train_images_torch.size)

train_labels_torch = torch.from_numpy(train_set_y).float()
print("shape of labels: ", len(train_labels_torch))

train_loader = torch.utils.data.TensorDataset(train_images_torch, train_labels_torch)
train_loader = torch.utils.data.DataLoader(train_loader, batch_size=batch_size, shuffle=True, num_workers=6)

cnn = CNN()
cnn.to(device)
print(cnn)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01, momentum=0.9)

start_time = time.time()
losses = []
for epoch in range(epochs):
    print("epoch number: ", epoch)
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
    # for some initial testing, I used the validation set because it is smaller.
    # for i, data in enumerate(val_loader, 0): 
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = rearrange(inputs, "(c b) h w -> b c h w", c = 1)
        # print("shape of inputs: ", inputs.shape)
        # print("shape of labels: ", labels.shape)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = cnn(inputs)

        # print("shape of outputs: ", outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        if i % 10 == 9:    # print every 2000 mini-batches
            # print("shape of inputs: ", inputs.shape)
            # print("shape of outputs: ", outputs.shape)
            # print("Here are the outputs: ", outputs)
            print(f'[epoch: {epoch + 1}, batch: {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            losses.append(running_loss)
            running_loss = 0.0

end_time = time.time()

train_time = end_time - start_time

print("Elapsed Training Time: ", datetime.timedelta(seconds = train_time))

print('Finished Training')